Pushpendra 23b1023-
Nischal 23b1024-
Nithin 23b0993

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [2]:
def data_split(df, split_ratio=0.9):
    data = np.asarray(df, dtype='float')
    np.random.shuffle(data)
    n = data.shape[1]
    split_index = int(split_ratio * data.shape[0])
    
    x_train = data[:split_index, 0:n-1]
    y_train = data[:split_index, n-1]
    x_test = data[split_index:, 0:n-1]
    y_test = data[split_index:, n-1]
    return x_train, y_train, x_test, y_test

In [3]:
def compute_B(x_train,y_train):
    x_transpose = np.transpose(x_train)
    x_transpose_dot_x = np.dot(x_transpose, x_train)
    inverse = np.linalg.inv(x_transpose_dot_x)
    B = np.dot(inverse, np.dot(x_transpose, y_train))
    return B

In [4]:
def cost(x,y,B):
    f = np.dot(x,B)
    cost = np.mean(np.abs(f - y))
    return cost

In [ ]:
train_path = 'train.csv'
test_path = 'test.csv'
submission_path = 'sample_submission.csv'

train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)

columns = ['id', 'Row#','MaxOfLowerTRange']
train_data = train_data.drop(columns=columns)
train_data[1] = train_data['fruitset']**2
train_data[2] = train_data['seeds']**2
train_data[3] = train_data['fruitmass']**2
train_data[4] = train_data['fruitset']**3
train_data[5] = train_data['seeds']**3
train_data[6] = train_data['fruitmass']**3
train_data[7] = train_data['fruitset']**4
train_data[8] = train_data['fruitset']*train_data['seeds']
train_data[9] = train_data['fruitset']**5

yeild = np.asarray(train_data['yield'])
train_data = train_data.loc[:, train_data.columns != 'yield']
train_data['yield'] = yeild
correlation_matrix = train_data.corr()

yield_correlation = correlation_matrix['yield'].sort_values(ascending=False)
print(yield_correlation)

In [ ]:
x_train, y_train, x_test, y_test = data_split(train_data)
x_train = np.hstack((np.ones((x_train.shape[0], 1)),x_train))
x_test = np.hstack((np.ones((x_test.shape[0], 1)), x_test))
B = compute_B(x_train, y_train)
cost(x_test,y_test,B)

In [7]:
test_data = pd.read_csv(test_path)
drop_columns = ['id', 'Row#', 'MaxOfLowerTRange']
test_data = test_data.drop(columns=drop_columns)

# Generate new features for test data, similar to the ones in train data
test_data[1] = test_data['fruitset']**2
test_data[2] = test_data['seeds']**2
test_data[3] = test_data['fruitmass']**2
test_data[4] = test_data['fruitset']**3
test_data[5] = test_data['seeds']**3
test_data[6] = test_data['fruitmass']**3
test_data[7] = test_data['fruitset']**4
test_data[8] = test_data['fruitset']*test_data['seeds']
test_data[9] = test_data['fruitset']**5

# test_data[8] = test_data['seeds']**4
# test_data[9] = test_data['fruitmass']**4
x = np.asarray(test_data)
x = np.hstack((np.ones((x.shape[0], 1)), x))
Y_test = np.dot(x,B)

# y_test = np.dot()

In [8]:
test_data1 = pd.read_csv(test_path)
test_data1 = test_data1[['id']]  # Keep only the 'id' column

# Add the predicted 'yield' values
test_data1['yield'] = Y_test
test_data1.to_csv('submission.csv', index=False)